# MGMGT 638-001 Assignment 2 
Author: Francho Garcia Takis <br>
Date: 13-11-2024

### Imports

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings("ignore")

### Data 

In [ ]:

data = pd.read_excel('../data/assignment2.xlsx')
data = data.drop(data.columns[0], axis=1) #drop the first col

In [3]:
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d') #convert date to datetime
data.head()

,ticker,date,pb,mom,ret
0,ACR,2023-01-06,0.2,-0.202085,0.084746
1,ACU,2023-01-06,1.0,-0.306748,-0.023573
2,AIRG,2023-01-06,1.6,-0.257761,0.062980
3,AIRT,2023-01-06,3.1,-0.032604,-0.017799
4,ARKR,2023-01-06,1.0,0.108092,0.044684


In [4]:
data['date'].unique()

array(['2023-01-06', '2023-01-13', '2023-01-20', '2023-01-27'],
      dtype=object)

Using the data for the first two dates (January 6 and 13), run a linear regression of ret on pb and mom (including an intercept)

In [5]:
train_one = data[data['date'] < '2023-01-14'] #Using only the 6th and 13th of Jan for fitting the regression model
reg_one = smf.ols(formula='ret ~ pb + mom', data=train_one).fit()
print(reg_one.summary())

                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.530
Date:                Tue, 12 Nov 2024   Prob (F-statistic):              0.219
Time:                        16:43:12   Log-Likelihood:                 181.98
No. Observations:                 200   AIC:                            -358.0
Df Residuals:                     197   BIC:                            -348.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0256      0.013      2.046      0.0

Using the coefficients from the regression and the pb and mom data on the third date (January 20), compute predicted returns

In [6]:
a_one, b_pb_one, b_mom_one = reg_one.params #Saving the regression coefficients

test_one = data[data['date'] == '2023-01-20'] #test set is the 20th of Jan

test_one['pred_ret'] = a_one + b_pb_one * test_one['pb'] + b_mom_one * test_one['mom'] #Calculate the predicted returns

Sort the stocks on January 20 on predicted returns.  Form two portfolios: (i) the 20 stocks with the highest predicted returns and (ii) the 20 stocks with the lowest predicted returns.  Calculate equally-weighted portfolio returns for the week ending January 20 for each of these portfolios.

In [7]:
test_one = test_one.sort_values('pred_ret', ascending=False) #sorting from high predicted returns to low

high_portfolio_one = test_one.head(20) #top 20 predicted returns portfolio
low_portfolio_one = test_one.tail(20)

In [8]:
print(f"High predicted return portfolio mean return: {high_portfolio_one['ret'].mean():.2%}")
print(f"Low predicted return portfolio mean return: {low_portfolio_one['ret'].mean():.2%}")

High predicted return portfolio mean return: 0.67%
Low predicted return portfolio mean return: -1.00%


Repeat 1-3, running the regression on the January 13 and January 20 data, and compute predicted returns and portfolio returns for January 27.

In [9]:
train_two = data[(data['date'] > '2023-01-06') & (data['date'] < '2023-01-21')] # Using the 13th and 20th of Jan for fitting the model

In [10]:
reg_two = smf.ols(formula='ret ~ pb + mom', data=train_two).fit()
print(reg_two.summary())

                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9374
Date:                Tue, 12 Nov 2024   Prob (F-statistic):              0.393
Time:                        16:43:12   Log-Likelihood:                 178.06
No. Observations:                 200   AIC:                            -350.1
Df Residuals:                     197   BIC:                            -340.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0200      0.013      1.520      0.1

In [11]:
a_two, b_pb_two, b_mom_two = reg_two.params #Saving the coefficients

test_two = data[data['date'] >= '2023-01-27'] # Using the 27th of Jan for the test set

test_two['pred_ret'] = a_two + b_pb_two * test_two['pb'] + b_mom_two * test_two['mom'] #Calculating the predicted returns

In [12]:
test_two = test_two.sort_values('pred_ret', ascending=False) #Sorting from high predicted returns to low

high_portfolio_two = test_two.head(20) #Taking the top 20 predicted returns
low_portfolio_two = test_two.tail(20)

print(f"High predicted return portfolio mean return: {high_portfolio_two['ret'].mean():.2%}")
print(f"Low predicted return portfolio mean return: {low_portfolio_two['ret'].mean():.2%}")

High predicted return portfolio mean return: 6.10%
Low predicted return portfolio mean return: 1.03%


Average the portfolio returns over the two dates, January 20 and January 27.  Did the stocks with the highest predicted returns beat the stocks with the lowest predicted returns?

In [13]:
#Average of the high and low portfolios across both test sets: 
print(f"Overall mean return of high predicted return portfolio: {(high_portfolio_one['ret'].mean() + high_portfolio_two['ret'].mean()) / 2:.2%}")
print(f"Overall mean return of low predicted return portfolio: {(low_portfolio_one['ret'].mean() + low_portfolio_two['ret'].mean()) / 2:.2%}")

Overall mean return of high predicted return portfolio: 3.39%
Overall mean return of low predicted return portfolio: 0.01%
